## capstone project 

#### Data for the project

The data consist in the different Borough and Neighborhood of Toronto, containing all the most important venues in the city.
to obtain the venues in Toronto we can use Foursquare API.

first we extract the Borough and Neighborhood scraping a wikipedia page using BeautifulSoup and then using Foursquare API we obtain all the venues of interest and the coordinates as well.

In [2]:
import pandas as pd
import requests
import csv
from bs4 import BeautifulSoup

res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))[0]
df.drop(df[df['Borough'] == 'Not assigned'].index , inplace=True)

df = df.reset_index(drop=True)
#print(df.columns)
sort = df.sort_values(by ='Postal code' , ascending=True)
sort = sort.reset_index(drop=True)
sort.shape

url = 'https://cocl.us/Geospatial_data'
data = pd.read_csv(url,sep=",") # use sep="," for coma separation. 
data.head()
sort2 = data.sort_values(by ='Postal Code' , ascending=True)
sort2.head(10)

lat = sort2["Latitude"]
lon = sort2["Longitude"]
sort = pd.concat([sort,lat], axis = 1)
sort = pd.concat([sort,lon], axis = 1)
sort

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,Malvern / Rouge,43.806686,-79.194353
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek,43.784535,-79.160497
2,M1E,Scarborough,Guildwood / Morningside / West Hill,43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,Kennedy Park / Ionview / East Birchmount Park,43.727929,-79.262029
7,M1L,Scarborough,Golden Mile / Clairlea / Oakridge,43.711112,-79.284577
8,M1M,Scarborough,Cliffside / Cliffcrest / Scarborough Village West,43.716316,-79.239476
9,M1N,Scarborough,Birch Cliff / Cliffside West,43.692657,-79.264848


 now we have most of the boroughs and Neighborhoods in Toronto
 
 

In [4]:
# set index for only a Borough for analisys 
toronto_data = sort[sort['Borough'] == 'Etobicoke'].reset_index(drop=True)
# eliminate 'Postcode' column
toronto_data=toronto_data.drop(['Postal code'], axis=1)
toronto_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Etobicoke,New Toronto / Mimico South / Humber Bay Shores,43.605647,-79.501321
1,Etobicoke,Alderwood / Long Branch,43.602414,-79.543484
2,Etobicoke,The Kingsway / Montgomery Road / Old Mill North,43.653654,-79.506944
3,Etobicoke,Old Mill South / King's Mill Park / Sunnylea /...,43.636258,-79.498509
4,Etobicoke,Mimico NW / The Queensway West / South of Bloo...,43.628841,-79.520999


In [5]:

CLIENT_ID = 'H002AVLCTRPE0XD5DXOEMVPIDZAKXTOZJ5CH5LGLVLNA0YFP' # your Foursquare ID
CLIENT_SECRET = 'ZLEEBGZOGGPKCNYEB1QYULYZBAXRC01XYDVPR4KLC2VKGUSF' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [6]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude'],
                                  )

New Toronto / Mimico South / Humber Bay Shores
Alderwood / Long Branch
The Kingsway / Montgomery Road / Old Mill North
Old Mill South / King's Mill Park / Sunnylea / Humber Bay / Mimico NE / The Queensway East / Royal York South East / Kingsway Park South East
Mimico NW / The Queensway West / South of Bloor / Kingsway Park South West / Royal York South West
Islington Avenue
West Deane Park / Princess Gardens / Martin Grove / Islington / Cloverdale
Eringate / Bloordale Gardens / Old Burnhamthorpe / Markland Wood
Westmount
Kingsview Village / St. Phillips / Martin Grove Gardens / Richview Gardens
South Steeles / Silverstone / Humbergate / Jamestown / Mount Olive / Beaumond Heights / Thistletown / Albion Gardens
Northwest


In [7]:
print(toronto_venues.shape)
toronto_venues.head()

(71, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,New Toronto / Mimico South / Humber Bay Shores,43.605647,-79.501321,LCBO,43.602281,-79.499302,Liquor Store
1,New Toronto / Mimico South / Humber Bay Shores,43.605647,-79.501321,New Toronto Fish & Chips,43.601849,-79.503281,Restaurant
2,New Toronto / Mimico South / Humber Bay Shores,43.605647,-79.501321,Domino's Pizza,43.601583,-79.500905,Pizza Place
3,New Toronto / Mimico South / Humber Bay Shores,43.605647,-79.501321,Delicia Bakery & Pastry,43.601403,-79.503012,Bakery
4,New Toronto / Mimico South / Humber Bay Shores,43.605647,-79.501321,Lucky Dice Restaurant,43.601392,-79.503056,Café


using Foursquare we can obtain all the data needed for this proyect, is our mayor source of information along with wikipedia and the Beautiful Soup library.
Beautiful Soup is a Python library for parsing HTML documents and is a great help when it comes to obtain data from a website.